In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import json
import os
from glob import glob
import numpy as np
import pandas as pd

from utils.parse_util import HTMLFileReader, LinkedInProfileParser
from utils.parse_pipeline import parse_profile
import webbrowser

In [90]:
LOG_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/parse_log/2000/'
PARSED_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/parsed_profiles/2000/'

### Log

In [91]:
def read_log_file():

    log_fpaths = glob(os.path.join(LOG_DIR, '*.json'))
    for fpath in log_fpaths:
        with open(fpath, 'r') as f:
            content = json.load(f)
        yield fpath, content

In [92]:
log_file_contents = read_log_file()

In [93]:
log_items = []

for fpath, log_content in log_file_contents:
    log_item = log_content.copy()
    log_item['empty_html_list'] = len(log_item['empty_html_list'])
    log_item['parse_error'] = len(log_item['parse_error'])
    log_items.append(log_item)

In [94]:
log_items = pd.DataFrame(log_items)

In [95]:
log_items.shape

(111, 4)

In [96]:
temp = log_items.sum()

In [97]:
temp

empty_html_list    0.000000e+00
parse_error        5.590000e+02
total_time         2.372714e+03
num_files          5.096381e+06
dtype: float64

In [98]:
(temp / temp['num_files']) * 100

empty_html_list      0.000000
parse_error          0.010969
total_time           0.046557
num_files          100.000000
dtype: float64

### Sample Log

In [ ]:
log_file = '122_log.json'
log_path = os.path.join(LOG_DIR, log_file)
log_path

In [ ]:
with open(log_path, 'r') as f:
    log_content = json.load(f)

In [ ]:
log_content.keys()

In [ ]:
log_content['parse_error']

In [ ]:
errored_files = list(log_content['parse_error'].keys())
errored_files

#### Error file check

In [ ]:
webbrowser.open('file://' + errored_files[4])

In [ ]:
html_reader = HTMLFileReader(errored_files[0])
html_reader.load()
clean_html = html_reader.get_clean_html_text()

In [ ]:
li_profile = LinkedInProfileParser(clean_html)
li_profile.parse()

In [ ]:
li

### Parsed Dirs

In [99]:
subdirs = os.listdir(PARSED_DIR)

In [100]:
len(subdirs)

111

### Parsed Profile

In [ ]:
parsed_file_paths = glob(os.path.join(PARSED_DIR, '*/*.json'))

In [ ]:
len(parsed_file_paths)

In [ ]:
sample_parsed_fp = np.random.choice(parsed_file_paths)

In [ ]:
with open(sample_parsed_fp, 'r') as f:
    sample_parse_content = json.load(f)

In [ ]:
sample_parse_content.keys()

In [ ]:
sample_parse_content['summary']